In [ ]:
import base64
import datetime
import json
import os
import random
import time

import scipy
from scipy import ndimage
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from pylab import *
from matplotlib.dates import date2num , DateFormatter
from PIL import Image

np.random.seed(1337)

%matplotlib inline
sns.set(font_scale=1.0)
rcParams['figure.figsize'] = 8, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

pd.options.display.max_colwidth = 1000

In [ ]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

# Load the Data

In [ ]:
raw_data = pd.read_json('tmp/raw_data.json')
d = pd.read_json('tmp/raw_data.json')
raw_data.index.name = 'id'
d = raw_data[['street_address']]
dd  = raw_data[['street_address', 'display_address']]
print len(d)

In [ ]:
d.dtypes

In [ ]:
dd.head()

In [ ]:
#dd[dd.street_address == dd.display_address].value_counts()
(dd.street_address == dd.display_address).value_counts()

## Number of words

Here I count numbers of abbreviations as words, so it's rather the number of tokens.

In [ ]:
# number of words
# daddr_num_words
def tokens(x):
    return len(x.split(' '))
d.loc[:, 'saddr_num_words'] = d.street_address.map(lambda x: tokens(x)).astype(float64)
d.saddr_num_words.value_counts().sort_index().plot(kind='bar')

## Does the address contain any numbers?

If the address contains number, then we can assume it's the exact address, not only the street or neighborhood name.

In [ ]:
def represents_int(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def has_number(x):
    return int(any([represents_int(w.strip()) for w in x.split(' ')]))
    
d.loc[:, 'saddr_has_number'] = d.street_address.map(lambda x: has_number(x))
d.head()

In [ ]:
d.saddr_has_number.value_counts().plot(kind='barh', figsize=(8,3))

## Does the address contain any ordinal numbers?


In [ ]:
import re
p = re.compile('[0-9]+[a-z]+')

def is_ordinal(w):    
    return p.match(w) != None

def has_ordinal_number(x):
    return int(any([is_ordinal(w.strip().lower()) for w in x.split(' ')]))
    
d.loc[:, 'saddr_has_ordinal'] = d.street_address.map(lambda x: has_ordinal_number(x))
d.head()

## Number of uppercase words

Note that there are a lot of addresses that contain a single-letter "word" which is uppercase, usually referring to East, West, etc
For example: E 78th St., W 68 Street.
I've excluded these single letter words because I think it's ok to treat them as normal.

In [ ]:
import re
uppercase_pattern = re.compile('^[A-Z]+$')


def is_uppercase(w):    
    return uppercase_pattern.match(w) != None

def uppercase_word_count(x):
    return len([w for w in x.split(' ') if is_uppercase(w.strip()) and len(w) > 1])
    
d.loc[:, 'saddr_num_upper_words'] = d.street_address.map(lambda x: uppercase_word_count(x)).astype(float64)
d.head()

In [ ]:
d[d.saddr_num_upper_words >2].head()

In [ ]:
d.saddr_num_upper_words.value_counts()

### Is street address the same as display address?

In [ ]:
d.loc[:, 'daddr_saddr_same'] = (dd.street_address == dd.display_address).map(int)
d.head()

# Num Apts with the Same Street Address